In [18]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
words = open('../names.txt', 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [20]:
vocab = sorted(set("".join(words)))
vocab = ['.'] + vocab
vocab_size = len(vocab)
vocab_size, vocab[:5]

(27, ['.', 'a', 'b', 'c', 'd'])

In [21]:
stoi = {s:i for i,s in enumerate(vocab)}
itos = {i:s for s,i in stoi.items()}
stoi

{'.': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [40]:
# init dataset
g = torch.Generator().manual_seed(821974)
block_size = 3

X,Y=[],[]
for w in words[:5]:
  context = [0] * block_size
  for ch in w+'.':
    X.append(context)
    Y.append(stoi[ch])
    context=context[1:] + [stoi[ch]]

X = torch.tensor(X)
Y = torch.tensor(Y)

print(X.shape, Y.shape)

n_embd = 2
n_hidden = 100
C  = torch.randn((vocab_size, n_embd)          , generator=g)
W1 = torch.randn((block_size*n_embd, n_hidden) , generator=g)
b1 = torch.randn(n_hidden                      , generator=g)
W2 = torch.randn((n_hidden, vocab_size)        , generator=g)
b2 = torch.randn(vocab_size                    , generator=g)

parameters = [C,W1,b1,W2,b2]
for p in parameters:
  p.requires_grad = True


torch.Size([32, 3]) torch.Size([32])


In [ ]:
# forward pass
emb = C[X]
hpreact = emb.view(-1, block_size*n_embd) @ W1 + b1
h = torch.tanh(hpreact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss

tensor([[1.1759, 0.2524],
        [1.1759, 0.2524],
        [1.1759, 0.2524]], grad_fn=<IndexBackward0>)

In [ ]:
# inference
for _ in range(5):
  context = [0] * block_size
  out=[]
  while True:
    emb = C[context]
    hpreact = emb.view(-1, block_size*n_embd) @ W1 + b1
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    probs = F.softmax(logits, dim=1)
    hit = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
    out.append(hit)
    if hit == 0:
      break
    context=context[1:]+[hit]
  print(''.join(itos[o] for o in out))

lpulbxqxqxqxqxqxqxq.
lpulqxqudwxalyx.
lpulbxqoyz.
lpulqxqxqxqxq.
lpulbxqoy.
